In [1]:
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob
import sys
import os
from scipy.interpolate import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import griddata
from skimage.filters import sobel


from spectral_analysis_functions import *
# Using OceanDataLab reader 
#from sentinel2_l1c import *
import numpy as np
from S2_read import *
from s2_angs import *
# bands=['B04','B08','B03','B02'];  



In [2]:
XML_File = find('MTD_TL.xml', S2path)
XML_File_DS=find('MTD_DS.xml',S2path)
if XML_File is None:
    print("Le fichier 'MTD_TL.xml' n'a pas été trouvé dans le répertoire spécifié:", S2path)
#else:
 #   (tile_id, AngleObs, AngleSun) = get_angleobs(XML_File)
#    print('Reading XML file for view and sun angles:', XML_File)
    
#boxi1=[4400,5200,3600,4400] # East West South North : this box is covered by another detector
# boxi1=[6800,7600,7000,7800] # East West South North : this box is covered by another detector
# [imgs1,NX1,NY1,nx1,ny1,dx1,dy1,offspec1,phitrig1,thetav1,loncenter1,latcenter1,detector,indexX,indexY]=S2_read(S2path,boxi1,bands)
#boxi=[5600,6400,3600,4400] # East West South North 
# boxi=[9000, 10000, 7000, 11000] # East West South North modif

[imgs,NX,NY,nx,ny,dx,dy,offspec,phitrig,thetav,loncenter,latcenter,detector,indexX,indexY]=S2_read(S2path,boxi,bands)

        

NameError: name 'S2path' is not defined

In [ ]:
print('lon,lat:',np.degrees(loncenter),np.degrees(latcenter))

nb=np.shape(bands)[0]


array1=np.squeeze(detector[1,:,:])
fig, ax = plt.subplots() 
im=ax.pcolor(indexX,indexY,array1)
fig.colorbar(im)
ax.set_title('detector index and FFT box location')
ax.set_xlabel('pixel index')
ax.set_ylabel('pixel index')
ax.axis('equal')
rect = mp.patches.Rectangle((boxi[0], boxi[2]), boxi[1]-boxi[0], boxi[3]-boxi[2], linewidth=2,
                         edgecolor='k', facecolor="none")
# rect1 = mp.patches.Rectangle((boxi1[0], boxi1[2]), boxi1[1]-boxi1[0], boxi1[3]-boxi1[2], linewidth=2,
#                          edgecolor='r', facecolor="none")
# ax.add_patch(rect1)

ax.add_patch(rect)
plt.show()

In [ ]:
# print('lon1,lat1:',loncenter,latcenter)
# print('lon,lat:',np.degrees(loncenter),np.degrees(latcenter))
# print('indexX','indexY',indexX,indexY)
# np.max(indexX)
# np.min(indexY)

# np.shape(array1)
# print('indexX',np.shape(indexX))
# print('indexY',np.shape(indexY))
# indexX

In [ ]:
# Conversion radian degree en gardant les float

loncenter=loncenter*180/(math.pi)
latcenter=latcenter*180/(math.pi)

# loncenter1=loncenter1*180/(math.pi)
# latcenter1=latcenter1*180/(math.pi)



In [3]:
# On determine le centre de la boîte en pixel
def centre_carre(x0,xy0,taille_boite):
    centrex=x0+taille_boite/2
    centrey=y0+taille_boite/2
    return centrex, centrey

x0=boxi[0]
y0=boxi[2]
cote=boxi[1]-boxi[0]
centre=centre_carre(x0,y0,cote)

NameError: name 'boxi' is not defined

In [4]:
# print('indexX',indexX)
indexX2, indexY2 = np.meshgrid(indexX, indexY)
print('shape indexX',np.shape(indexX))
print('shape indexY',np.shape(indexY))
print('shape array1',np.shape(array1))


# Aplatir les grilles et les valeurs pour utiliser dans griddata
points = np.column_stack((indexX.ravel(), indexY.ravel()))
values = array1.ravel()

# Interpolation
value = griddata(points, values, centre, method='linear')
value = value.item()  # Extraire la valeur si c'est un tableau 1D avec un seul élément
print(value)

NameError: name 'indexX' is not defined

In [5]:
mapping = {
    1: 'D01',
    2: 'D02',
    3: 'D03',
    4: 'D04',
    5: 'D05',
    6: 'D06',
    7: 'D07',
    8: 'D08',
    9: 'D09',
    10: 'D10',
    11: 'D11',
    12: 'D12'
}
if round(value) in mapping:
    Det = mapping[round(value)]
else:
    # Si la valeur arrondie n'est pas dans le dictionnaire, attribuez une valeur par défaut ou gérez l'erreur selon votre cas d'utilisation
    Det = "Valeur non trouvée"
print(Det)

detector_find=int(value)

NameError: name 'value' is not defined

In [6]:
datastrip_dir = os.path.join(S2path, "DATASTRIP")

# Liste des éléments dans le répertoire DATASTRIP
datastrip_contents = os.listdir(datastrip_dir)

# Filtrer les répertoires dans datastrip_contents
directories = [item for item in datastrip_contents if os.path.isdir(os.path.join(datastrip_dir, item))]

# Vérifier qu'il y a exactement un répertoire dans DATASTRIP
if len(directories) == 1:
    # Si un seul répertoire est présent, construire le chemin vers MTD_DS.xml
    xml_path = os.path.join(datastrip_dir, directories[0], "MTD_DS.xml")
    print(xml_path)
else:
    print("Il n'y a pas exactement un seul répertoire dans DATASTRIP.")

NameError: name 'S2path' is not defined

In [7]:
# Renaud Binet's Algo

# !python main_computeDT.py -lon float\(np.degrees\(loncenter1\)\) -lat float\(np.degrees\(latcenter1\)\) -src B2 -dst B4 -det D11 -sat B -eph /home/acuevas/Bureau/S2_ALEXANDRA/S2B_MSIL1C_20230430T190919_N0509_R056_T10SBG_20230430T211611.SAFE/DATASTRIP/DS_2BPS_20230430T211611_S20230430T191523/MTD_DS.xml
# %run main_computeDT.py -lon float(np.degrees(loncenter1)) -lat float(np.degrees(latcenter1)) -src B2 -dst B4 -det D12 -sat B -eph /home/acuevas/Bureau/S2_ALEXANDRA/S2B_MSIL1C_20230430T190919_N0509_R056_T10SBG_20230430T211611.SAFE/DATASTRIP/DS_2BPS_20230430T211611_S20230430T191523/MTD_DS.xml

# dt1=!python main_computeDT.py -lon {loncenter} -lat {latcenter} -src B2 -dst B4 -det {Det} -sat B -eph /home/acuevas/Bureau/S2_ALEXANDRA/S2B_MSIL1C_20230430T190919_N0509_R056_T10SBG_20230430T211611.SAFE/DATASTRIP/DS_2BPS_20230430T211611_S20230430T191523/MTD_DS.xml
# dt1=!python main_computeDT.py -lon {loncenter} -lat {latcenter} -src B2 -dst B4 -det D12 -sat B -eph /home/acuevas/Bureau/S2_ALEXANDRA/S2B_MSIL1C_20230430T190919_N0509_R056_T10SBG_20230430T211611.SAFE/DATASTRIP/DS_2BPS_20230430T211611_S20230430T191523/MTD_DS.xml

dt1=!python main_computeDT.py -lon {loncenter} -lat {latcenter} -src B2 -dst B4 -det {Det} -sat {Sat} -eph {xml_path}
print('dt=',dt1[0])

dt= usage: main_computeDT.py [-h] -lon LONGITUDE -lat LATITUDE -src SRC_BAND -dst


In [8]:
# # ## Calcul du dt par le tableau dans Yurovsakya et al 2019

# data = np.loadtxt('Inter_detector_time_lag_yetal2019.csv', delimiter=',')
# inter_bandB2=data[1,int(detector_find-1)]
# inter_bandB4=data[3,int(detector_find-1)]
# # print(inter_bandB2)
# # print(inter_bandB4)

# dt1=inter_bandB4-inter_bandB2

In [9]:
# normalizes the image values
img1=imgs[0,:,:]/np.median(imgs[0,:,:].flatten())
img2=imgs[3,:,:]/np.median(imgs[3,:,:].flatten())
# plt.imshow(np.flipud(np.transpose(img1)),vmin=0.8, vmax=1.5) #edit your vmin, vmax and cmap if you don't like greyscale colormap
# plt.colorbar()
# plt.show()
# plt.imshow(np.flipud(np.transpose(img2)),vmin=0.8, vmax=1.5) #edit your vmin, vmax and cmap if you don't like greyscale colormap
# plt.colorbar()
# plt.show()

NameError: name 'imgs' is not defined

### Filtre du déferlement

In [10]:
gradient_sobel1 = sobel(img1)
gradient_sobel2=sobel(img2)

# # Visualiser le gradient
# plt.figure(figsize=(8, 4))
# plt.imshow(gradient_sobel1, cmap="gray", origin="upper")
# plt.colorbar(label="Amplitude du gradient")
# plt.title("Détection des bords par gradient de Sobel")
# plt.show()

# from scipy.ndimage import gaussian_filter

# img_smooth = gaussian_filter(img1, sigma=1)
# gradient_sobel1 = sobel(img_smooth)
# img_smoot2 = gaussian_filter(img2, sigma=1)
# gradient_sobel2 = sobel(img_smoot2)


threshold = np.percentile(gradient_sobel1, 95)  # Définir un seuil automatique
threshold2 = np.percentile(gradient_sobel2, 95)  # Définir un seuil automatique

mask = gradient_sobel1 < threshold  # Garder les zones avec un faible gradient
mask2 = gradient_sobel2 < threshold2  # Garder les zones avec un faible gradient

# Appliquer le masque à l'image originale
filtered_image1 = np.where(mask, img1, np.nan)  # Remplace les pixels masqués par NaN
filtered_image2 = np.where(mask2, img2, np.nan)  # Remplace les pixels masqués par NaN

# # Affichage du masque et de l'image filtrée
# fig, axs = plt.subplots(1, 2, figsize=(12, 6))
# axs[0].imshow(mask, cmap="gray")
# axs[0].set_title("Masque des zones valides")
# axs[1].imshow(filtered_image1, cmap="gray")
# axs[1].set_title("Image après filtrage des zones turbulentes")
# plt.show()

NameError: name 'img1' is not defined

In [11]:
#img2=img1[0:400,0:800]
#(Eta,Eta_all,kx2,ky2,dkx,dky)=FFT2D_one_array(img1,10,10,16,isplot=0)
ntile=16 # le 10/01/25
# ntile=8
(Eta,Etb,ang,angstd,coh,crosr,phases,kx2,ky2,dkx,dky)=FFT2D_two_arrays_nonan(filtered_image1,filtered_image2,10,10,ntile,isplot=0)
print(np.shape(Eta))
print(np.shape(kx2))
print(np.shape(ky2))


phiblind1=phitrig[0]+90.  # blind azimuth for first band
phiblind2=phitrig[1]+90.  # blind azimuth for 3rd band
phiblind3=phitrig[2]+90.  # blind azimuth for 3rd band
kN=50
xkblind=[-1.4*kN*np.sin(np.radians(phiblind2)) , 1.4*kN*np.sin(np.radians(phiblind2))]
ykblind=[-1.4*kN*np.cos(np.radians(phiblind2)) , 1.4*kN*np.cos(np.radians(phiblind2))]


X = np.arange(0,nx*dx,dx) # from 0 to (nx-1)*dx with a dx step
Y = np.arange(0,ny*dy,dy)

fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
_=plt.gca().set_aspect('equal', adjustable='box')

# warning: we use an array img1 that is written with X in the first (number 0) dimension and Y in the second
#          but pcolormesh uses the opposite order: hence the use of transpose 

# im=axs[0].pcolormesh(X/1000,Y/1000,np.transpose(img1),cmap='seismic',norm = mcolors.Normalize(vmin=0.5, vmax=2))
# _=plt.colorbar(im,ax=axs[0],label='S2 image band B04')
# _=axs[0].set_xlabel('X [km]')
# _=axs[0].set_ylabel('Y [km]')
# _=axs[0].set_title('Surface')

# im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Eta)),norm = mcolors.Normalize(vmin=-30, vmax=20))
# _=plt.colorbar(im,ax=axs[1],label='dB')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[1].set_title('Spectrum')
# _=plt.tight_layout()

# print('variance 1:',np.var(img1.flatten()), 'sum of spectrum:',np.sum(Eta.flatten())*dkx*dky )
# print('variance 2:',np.var(img2.flatten()), 'sum of spectrum:',np.sum(Etb.flatten())*dkx*dky )


# fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
# _=plt.gca().set_aspect('equal', adjustable='box')

# im=axs[0].pcolormesh(kx2*1000,ky2*1000,coh,norm = mcolors.Normalize(vmin=0, vmax=1))
# _=plt.colorbar(im,ax=axs[0],label=' ')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[0].set_title('coherence B02-B04')

# im=axs[1].pcolormesh(kx2*1000,ky2*1000,np.degrees(ang),cmap='seismic',norm = mcolors.Normalize(vmin=-180, vmax=180))
# _=plt.colorbar(im,ax=axs[1],label='degrees')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[1].set_title('cross-spectrum phase')
# _=plt.tight_layout()

# fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
# _=plt.gca().set_aspect('equal', adjustable='box')

# im=axs[0].pcolormesh(kx2*1000,ky2*1000,10*np.log10((crosr)),norm = mcolors.Normalize(vmin=-30, vmax=20))
# _=plt.colorbar(im,ax=axs[0],label=' ')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[0].set_title('real part of X-spec')

# im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Eta)),norm = mcolors.Normalize(vmin=-30, vmax=20))
# _=plt.colorbar(im,ax=axs[1],label='dB')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[1].set_title('Spectrum')
# _=plt.tight_layout()


NameError: name 'filtered_image1' is not defined

In [12]:
# print(coh[34,14],coh[14,34])

In [13]:
# ang[0,:]

In [14]:
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20 #Jusque là les dimmensions sont ok avec Matlab
kncpk=kn/(2*np.pi)*1000

# # Let's now divide by MTF ... 
# # 
# print('G values:',G4,G8,np.shape(dir2a-phiblind2))
# MTF=1/(G4*kn*np.sin(np.radians(dir2a-phiblind2)))**2
# J2=np.where((coh < 0.01) | (MTF > 300) )
# MTF[J2]=0
# Etb=Eta
# Etb[J2]=0
# Ek=crosr*MTF

# fig,axs=plt.subplots(1,2,figsize=(20,7))#,sharey=True,sharex=True)
# _=plt.gca().set_aspect('equal', adjustable='box')

# # warning: we use an array img1 that is written with X in the first (number 0) dimension and Y in the second
# #          but pcolormesh uses the opposite order: hence the use of transpose 

# im=axs[0].pcolormesh(kx2*1000,ky2*1000,10*np.log10((MTF)),norm = mcolors.Normalize(vmin=-200, vmax=30))
# _=plt.colorbar(im,ax=axs[0],label='MTF (dB)')
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[0].set_title('MTF')

# im=axs[1].pcolormesh(kx2*1000,ky2*1000,10*np.log10((Ek)),norm = mcolors.Normalize(vmin=-20, vmax=40))
# _=plt.colorbar(im,ax=axs[1],label='dB')
# _=plt.plot(xkblind,ykblind,color='k',linestyle='--',linewidth=2)
# _=axs[1].set_xlabel('k_x [cycles / km]')
# _=axs[1].set_ylabel('k_y [cycles / km]')
# _=axs[1].set_xlim([-10,10])
# _=axs[1].set_ylim([-10,10])
# _=axs[1].set_title('Spectrum')
# _=plt.tight_layout()

# varB=np.sum(Etb.flatten())*dkx*dky
# sumE=np.sum(Ek.flatten())*dkx*dky


# print('sum of spectrum:',sumE,sqrt(sumE), 4*sqrt(sumE) )
# print(np.degrees(ang))


NameError: name 'kx2' is not defined

In [15]:

# print('dir2a:',np.shape(dir2a))
# print('kn:', np.shape(kn),max(kn.flatten()))

In [16]:
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20

# dirC1=-50
# dirC2=-40

# dirC1=130
# dirC2=140
# dirC1=-160
# dirC2=-150

# dirC1=-40
# dirC2=-30
dirC1=-100
dirC2=-90

dt=float(dt1[0])
# dt=dt1
J =np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) & (np.degrees(angstd)  < 60) )
J2=np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) )
dispt=np.sqrt(9.81/kn)
Cimg=ang/kn/dt
Cstd=angstd/kn/abs(dt)

# plt.plot(kn[J2]/(2*pi)*1000,dispt[J2],'kx',linewidth=3,label='Airy theory') 
# plt.plot(kn[J2]/(2*pi)*1000,-Cimg[J2],'ro',linewidth=1,label='from S2 phase')
# plt.plot(kn[J]/(2*pi)*1000,-Cimg[J],'bo',linewidth=1,label='from image X-spectra') 
# plt.grid(True)
# plt.xlabel('k (cycles / km)')
# plt.ylabel('phase speed (m/s)')
# plt.legend()

NameError: name 'kx2' is not defined

In [17]:
# print('type(ang)=',type(ang))
# print('type(kn)=',type(kn))
# print('type(dt)=',type(dt))


In [18]:
# plt.errorbar(kn[J2]/(2*np.pi)*1000,-Cimg[J2],yerr=Cstd[J2]/16,fmt='.r',capsize=5,label='from S2 phase')
# plt.errorbar(kn[J]/(2*np.pi)*1000,-Cimg[J],yerr=Cstd[J]/16,fmt='.b',capsize=5,label='from image X-spectra')
# plt.plot(kn[J2]/(2*np.pi)*1000,dispt[J2],'kx',linewidth=3,label='Airy theory') 
# plt.grid(True)
# plt.axis([8,50,4,14])
# plt.xlabel('k (cycles / km)')
# plt.ylabel('phase speed (m/s)')
# plt.legend()
# plt.show()

In [19]:
##PLot current velocity
# Cimg1=Cimg[J]
# Cimg2=Cimg[J2]
# plt.clf()
# plt.grid(True)
# plt.errorbar(kn[J2]/(2*np.pi)*1000,-Cimg2,yerr=Cstd[J2]/16,fmt='.r',capsize=5,label='from S2 phase')
# plt.errorbar(kn[J]/(2*np.pi)*1000,-Cimg1,yerr=Cstd[J]/16,fmt='.b',capsize=5,label='from image X-spectra')
# plt.plot(kn[J2]/(2*np.pi)*1000,dispt[J2],'kx',linewidth=3,label='Airy theory')
# plt.axis([8, 50, -1.2, 1.2])
# plt.xticks(np.linspace(0,50,6))
# plt.rcParams['font.size']=14
# plt.legend()
# plt.show()

In [20]:
d2r=np.pi/180
phase_o = abs(dt) * np.sqrt(9.81 * kn)  # phase from linear theory (deep water)
phase_diff=ang-phase_o*np.sign(ang)# phase_diff should be equal to kx.Ux + ky.Uy ...

# FA modifications for later  : manages the possible sign flip : uses the +phase or -phase ... 
# Better trick from Nicolas: work with complex numbers and divide the complex phase by exp(-i*omega*t)  ... 

Clin=np.sqrt(9.81/kn)
philinp= (Clin*kn*dt+pi)%(2*np.pi)-np.pi   # % is the mod operator: np.remainder 
philinm= (-Clin*kn*dt+pi)%(2*pi)-np.pi
Cimg0=(ang-philinm)/kn/dt;
Cimg2=(ang-philinp)/kn/dt;
signchoice=np.where(abs(Cimg2) < abs(Cimg0));
Cimg0[signchoice]=Cimg2[signchoice];
Cimgp=Cimg0;
kfac=1000/(2*np.pi)
J3=np.where( (coh < 0.64) | (kn < 10/kfac) ) #; % | dir2a < 0); % | dir2a > 90);
Cimgp[J3]=numpy.nan;

# Cimgp is kx*U+ky*V 

# # Calcul de la phase à partir de la théorie linéaire (eaux profondes)
# phase_o = abs(dt) * np.sqrt(9.81 * kn)

# # Calcul des vitesses de phase
# Clin = np.sqrt(9.81 / kn)

# # Phases linéaires modifiées
# philinp = (Clin * kn * dt + np.pi) % (2 * np.pi) - np.pi
# philinm = (-Clin * kn * dt + np.pi) % (2 * np.pi) - np.pi

# # Calcul des Cimg0 et Cimg2
# Cimg0 = (ang - philinm) / (kn * dt)
# Cimg2 = (ang - philinp) / (kn * dt)

# # Choix du signe basé sur la plus petite valeur absolue
# signchoice = np.where(np.abs(Cimg2) < np.abs(Cimg0))
# Cimg0[signchoice] = Cimg2[signchoice]

# # Résultat de Cimg
# Cimgp = Cimg0

# # Facteur de conversion
# kfac = 1000 / (2 * np.pi)

# # Conditions pour définir les valeurs de Cimgp à NaN
# J3 = np.where((coh < 0.64) | (kn < 10 / kfac))
# Cimgp[J3] = np.nan

# # Utilisation de nombres complexes pour diviser la phase par exp(-i * omega * t)
# omega = np.sqrt(9.81 * kn)  # fréquence angulaire
# t = dt  # temps considéré

# # Phase complexe initiale (exemple)
# phase_complex_initiale = np.exp(1j * ang)

# # Phase complexe divisée par exp(-1j * omega * t)
# phase_complex_modifiee = phase_complex_initiale / np.exp(-1j * omega * t)

# # Résultat final
# phase_modifiee = np.angle(phase_complex_modifiee)

# # Utilisation de phase_modifiee dans le calcul de phase_diff
# phase_diff = ang - phase_modifiee * np.sign(ang)



# k_min_fit = 15 # in cpk
# 17/05/2024
k_min_fit=15  #in cpk

# k_max_fit = 35 # in cpk
# 21/05/2024
k_max_fit=35 # in cpk
std_max = 60*d2r
# id_fit = np.where((angstd / d2r < std_max) & (kncpk >= k_min_fit) & (kncpk <= k_max_fit))  # & phase > 0

# id_fit = np.where((angstd.flatten()  < std_max)  & (kncpk.flatten() >= k_min_fit) & (kncpk.flatten() <= k_max_fit) & (coh.flatten()>=0.6))[0]  # & phase > 0
id_fit = np.where((angstd.flatten()  < std_max)  & (kncpk.flatten() >= k_min_fit) & (kncpk.flatten() <= k_max_fit))[0]  # & phase > 0

# id_fit = np.where((angstd / d2r < std_max) & (kncpk >= k_min_fit) & (kncpk <= k_max_fit))[0]  # & phase > 0
angstd1=angstd.flatten()
phase_diff1=phase_diff.flatten()
weight = ntile**2 / (angstd1[id_fit] ** 2)/dt**2
kx1=kx2.flatten()
ky1=ky2.flatten()
K_x_fit = kx1[id_fit]
K_y_fit = ky1[id_fit]
Y = phase_diff1[id_fit] / dt
C=np.diag(weight)
A=(np.column_stack((K_x_fit ,K_y_fit)))*(2*np.pi)  # these are now in rad / m 

# X=np.linalg.lstsq(A, Y, rcond='warn')
# print('U and V components of current :',X[0])

# Pondération 

ATC=np.matmul(A.T,C)
D=np.matmul(ATC,A)
#print('A:',A)
DI=np.linalg.inv(D)
X=np.matmul(DI,np.matmul(ATC,Y))
print('U and V components of current :',X)


sigU=np.sqrt(abs(DI))
print('uncertainties for U and V:',sigU[0,0],sigU[1,1])

# X1=np.linalg.lstsq(A2[0,:,:],Y,rcond=None)[0]
# X2=np.linalg.lstsq(A2[1,:,:],Y,rcond=None)[0]



fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax.scatter(K_x_fit, K_y_fit)
ax.set_title('selected spectral components for fit')
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(K_x_fit, K_y_fit, Y)




NameError: name 'dt' is not defined